In [1]:
from PIL import Image
import glob
import matplotlib.pyplot as plt

def resize_images(img_path):
  images = glob.glob(img_path + "/*.jpg")

  print(len(images), " images to be resized.")

  # 파일마다 28x28 사이즈로 바꿔 저장한다.
  target_size = (28, 28)
  for img in images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img, "JPEG")

  print(len(images), " images resized.")

# 학습 데이터 로드 및 rsize
image_dir_path = "/content/rock_scissor_paper/scissor"
resize_images(image_dir_path)
print("가위 이미지 resize 완료!")
image_dir_path = "/content/rock_scissor_paper/rock"
resize_images(image_dir_path)
print("바위 이미지 resize 완료!")
image_dir_path = "/content/rock_scissor_paper/paper"
resize_images(image_dir_path)
print("보 이미지 resize 완료!")

# test 데이터 로드 및 resize
image_dir_path = "/content/rock_scissor_paper/test/scissor"
resize_images(image_dir_path)
print("가위 이미지 resize 완료!")
image_dir_path = "/content/rock_scissor_paper/test/rock"
resize_images(image_dir_path)
print("바위 이미지 resize 완료!")
image_dir_path = "/content/rock_scissor_paper/test/paper"
resize_images(image_dir_path)
print("보 이미지 resize 완료!")

300  images to be resized.
300  images resized.
가위 이미지 resize 완료!
300  images to be resized.
300  images resized.
바위 이미지 resize 완료!
300  images to be resized.
300  images resized.
보 이미지 resize 완료!
100  images to be resized.
100  images resized.
가위 이미지 resize 완료!
100  images to be resized.
100  images resized.
바위 이미지 resize 완료!
100  images to be resized.
100  images resized.
보 이미지 resize 완료!


In [2]:
import numpy as np

def load_data(img_path, number_of_data=900):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = "/content/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

image_dir_path = "/content/rock_scissor_paper/test"
(x_test, y_test)=load_data(image_dir_path)
x_test_norm = x_test/255.0

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 900 입니다.
학습데이터(x_train)의 이미지 개수는 300 입니다.
x_train shape: (900, 28, 28, 3)
y_train shape: (900,)


In [3]:
import tensorflow as tf
from tensorflow import keras

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(256, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(256, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

print('Model에 추가된 Layer 개수: ', len(model.layers))

model.summary()

Model에 추가된 Layer 개수:  9
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 256)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 256)         590080    
                                                                 
 max_pooling2d_2 (MaxPooling  (N

In [4]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train_norm, y_train, epochs=15)

Epoch 1/15
29/29 [==============================] - 11s 19ms/step - loss: 0.9256 - accuracy: 0.5556
Epoch 2/15
29/29 [==============================] - 0s 12ms/step - loss: 0.5109 - accuracy: 0.7533
Epoch 3/15
29/29 [==============================] - 0s 13ms/step - loss: 0.3818 - accuracy: 0.8100
Epoch 4/15
29/29 [==============================] - 0s 12ms/step - loss: 0.3103 - accuracy: 0.8722
Epoch 5/15
29/29 [==============================] - 0s 12ms/step - loss: 0.3301 - accuracy: 0.8544
Epoch 6/15
29/29 [==============================] - 0s 12ms/step - loss: 0.2636 - accuracy: 0.8978
Epoch 7/15
29/29 [==============================] - 0s 13ms/step - loss: 0.1831 - accuracy: 0.9322
Epoch 8/15
29/29 [==============================] - 0s 12ms/step - loss: 0.1567 - accuracy: 0.9500
Epoch 9/15
29/29 [==============================] - 0s 12ms/step - loss: 0.1279 - accuracy: 0.9544
Epoch 10/15
29/29 [==============================] - 0s 12ms/step - loss: 0.1335 - accuracy: 0.9422
Epoch 11

In [5]:
test_loss, test_accuracy = model.evaluate(x_test_norm,y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

29/29 - 0s - loss: 3.0710 - accuracy: 0.8156 - 337ms/epoch - 12ms/step
test_loss: 3.0709657669067383 
test_accuracy: 0.8155555725097656
